In [2]:
# Custom Modules import
from util.data_loader import load_competitions, load_matches
from util.league_table import generate_league_table
from util.elo_calculator import initialize_team_stats, calculate_elo_ratings, loss_from_comparing_tables, elo_grid_search

from statsbombpy import sb
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Nuanced Team Strength Model

### Legend:  
🔴- to do <br>
🟠- in progress <br>
🟢- done <br>


### Used Features to quantify Team Stregnth:
- 🔴 Possession
- 🟢Shots on Goal/ 🔴 on Target
- 🟢Passes 
- 🟠Duels


### Other Features used to predict Match Outcomes:
- 🔴 Last 5 matches
- 🔴 Starting XI
- 🔴 Stakes of the match 
- 🔴 Injured/Penalized players
- 

In [3]:
events = sb.events(match_id=3890259)
events.shape

C:\Users\Max\AppData\Roaming\Python\Python312\site-packages\statsbombpy\api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


(3691, 91)

In [61]:
print('------------distinct_types----------------')
print(events['type'].unique())
print('------------pass_outcome----------------')
print(events['pass_outcome'].unique())
print('------------pass_type-------------')
print(events['pass_type'].unique())
print('------------pass_height----------------')
print(events['pass_height'].unique())

------------distinct_types----------------
['Starting XI' 'Half Start' 'Pass' 'Pressure' 'Ball Receipt*' 'Carry'
 'Duel' 'Clearance' 'Ball Recovery' 'Interception' 'Dispossessed'
 'Miscontrol' 'Block' 'Foul Committed' 'Foul Won' 'Shot' 'Goal Keeper'
 'Shield' '50/50' 'Dribble' 'Dribbled Past' 'Injury Stoppage' 'Player Off'
 'Player On' 'Half End' 'Error' 'Substitution' 'Tactical Shift']
------------pass_outcome----------------
[nan 'Incomplete' 'Out' 'Unknown' 'Pass Offside']
------------pass_type-------------
[nan 'Kick Off' 'Throw-in' 'Recovery' 'Free Kick' 'Goal Kick' 'Corner'
 'Interception']
------------pass_height----------------
[nan 'Ground Pass' 'High Pass' 'Low Pass']


In [ ]:
all_shots = events[events['type'] == 'Shot'].dropna(axis=1, how='all').groupby(['team']).size().reset_index(name='Shots').sort_values(['team','Shots'], ascending=False)

goals =  events[(
    events['type'] == 'Shot') & 
    (events['shot_outcome'] == 'Goal') 
].dropna(axis=1, how='all'
).groupby(['team']
).size(
).reset_index(name='Goals'
).sort_values(['team','Goals'], ascending=False)

#Still wrong (Duels: 128,  Source: Kicker https://www.kicker.de/bayern-gegen-hsv-2015-bundesliga-2854915/spieldaten)
duels = duels_per_team = events[(events['type'] == 'Duel')].dropna(axis=1, how='all').shape[0]

won_duels = events[
    (events['type'] == 'Duel') |
     #(events['duel_outcome'] == 'Success In Play') |
    (events['type'] == '50/50')
].dropna(axis=1, how='all')



# Calculate total passes
total_passes = (events[events['type'] == 'Pass']
               .groupby('team')
               .size()
               .reset_index(name='total_passes'))

# Calculate completed passes (successful passes with no outcome)
completed_passes = (events[(events['type'] == 'Pass') & 
                         (events['pass_outcome'].isnull())]
                  .groupby('team')
                  .size()
                  .reset_index(name='completed_passes'))


,team,Shots
1,Hamburger SV,5
0,Bayern Munich,22


In [62]:
# Joining all Stat Dataframes together
team_stats = pd.merge(completed_passes, total_passes, on='team')
# Calculate completion percentage
team_stats['completion_percentage'] = (team_stats['completed_passes'] / 
                                     team_stats['total_passes']) * 100

team_stats = pd.merge(team_stats, all_shots, on='team')
team_stats['duels'] = duels
team_stats = pd.merge(team_stats, goals, on='team', how='outer').fillna(0)



team_stats.head()

,team,completed_passes,total_passes,completion_percentage,Shots,duels,Goals
0,Bayern Munich,711,797,89.209536,22,76,5.0
1,Hamburger SV,150,240,62.500000,5,76,0.0
